In [1]:

import torch

In [2]:
from agents.DQNagent import *
from agents.PPOagent import *
from gameboy_env import *
import sys
sys.path.append('models')
from models.dqn import *

In [3]:
print(torch.cuda.is_available())
def train_agent(worker_id, episodes, config_params):
    # Initialize environment, agent, and config here using config_params
    env = GameBoyEnv("roms/Super Mario Land (JUE) (V1.1) [!].gb", observation_shape=(84,84), window_type="human")
    config = Config(env, DQN, DQN, **config_params)
    agent = DQNAgent(config)
    
    # Training the agent
    agent.train(episodes, discount_step_interval=300, reward_discount_factor=.989)
    print(f"Worker {worker_id} finished training.")

from multiprocessing import Process

# Define the number of processes/workers you want to run in parallel
num_workers = 2  # For example, running 4 parallel training sessions
episodes_per_worker = 7500  # Total episodes for each worker

# Configuration parameters that are common across workers, could be adjusted per worker if needed
config_params1 = {
    "checkpoint_subfolder": "limited_rom_encoder_3dof_lineardecay_bs32",
    "epsilon": .92,
    "batch_size": 32,
    "n_ticks_to_skip": 3
}

config_params2 = {
    "checkpoint_subfolder": "limited_rom_encoder_3dof_lineardecay_bs64",
    "epsilon": 1,
    "batch_size": 64,
    "n_ticks_to_skip": 3
}

# Create and start a process for each worker
processes = []
for worker_id in range(num_workers):
    # Determine the configuration parameters based on worker_id
    config_params = config_params1 if worker_id % 2 == 0 else config_params2
    
    # Create and start the process with the selected configuration
    p = Process(target=train_agent, args=(worker_id, episodes_per_worker, config_params))
    processes.append(p)
    p.start()

# Wait for all processes to complete
for p in processes:
    p.join()


print("All workers have completed their training.")


True
All workers have completed their training.
